For more information about how to setup google sheets api:

Portuguese: https://developers.google.com/sheets/api/quickstart/python?hl=pt-br

English: https://developers.google.com/sheets/api/quickstart/python

``` python
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

In [5]:
import pandas as pd
from gsheets_utils import *

In [6]:
gs_client = GoogleSheetsClient(credentials_file='credentials.json')


# CONTROLE DE PROJETO

In [9]:
csc_df = gs_client.get_sheet_data_as_dataframe(
    spreadsheet_id='1Q-P0QxPWxH_rCLslEFetiht9-SmmGtwyd5TPsm2HAp0',
    range_name='Tarefas!A1:T1000'
)

In [10]:
csc_data = gs_client.get_sheet_data(
    spreadsheet_id='1Q-P0QxPWxH_rCLslEFetiht9-SmmGtwyd5TPsm2HAp0',
    range_name='Tarefas!A1:T1000'
)

In [36]:
pmo_data = gs_client.get_sheet_data(
    spreadsheet_id='1aD6NWbiaIS1H4N09nkimKGeKJeD9ux2coW9aY2ZM-rc',
    range_name='Tarefas!A1:T1000'
)

In [35]:
tlog_data = gs_client.get_sheet_data(
    spreadsheet_id='128DTCwkR6DcthbwQzGmlGfAUnv6KuNidlE4_RWhfiXY',
    range_name='Tarefas!A1:T3000'
)

In [40]:
engagement_survey_data = gs_client.get_sheet_data(
    spreadsheet_id='1Rxo642UduYjf1OvHmvgXS5UN4OBwXK1nfOOWXMcYBrA',
    range_name='Respostas ao formulário 1!A1:P100'
)

# SNOWFLAKE

Snowflake-Snowpark-Python compatibility issue. Need to downgrade snowflake-snowpark-python to 1.9.0 (for Python 3.11, 3.10, 3.9, 3.8) and install snowflake connector using below command,( as per https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-pandas#installation)

´´´python
pip install snowflake-connector-python[pandas]
´´´

In [11]:
from config import *
from snowpark_utils import *
from snowflake.snowpark.session import Session
# from snowflake.connector.pandas_tools import write_pandas


In [12]:
session = ConnectSession()

Starting connection...
Connected to Schema: "SANDBOX"."DEV_IVON_GARCIA"


## Projects

1. Data Preprocessing:
* Fix Incomplete Rows: Corrects rows that have fewer columns than expected by padding them with empty strings.
* Clean Column Names: Replaces spaces and special characters in column names to ensure compatibility with Snowflake.

2. Creating the Pandas DataFrame: Uses the cleaned column names and fixed data.
    
3. Converting to Snowpark DataFrame: Converts the Pandas DataFrame to a Snowpark DataFrame.
    
4. Saving the DataFrame as a Table: Saves the DataFrame to Snowflake, using overwrite mode to replace any existing table with the same name.
    
5. Display Confirmation: Prints the first few rows of the newly created table to confirm successful creation.

In [34]:
def create_table(session, data, table_name):
    """
    Creates a table in Snowflake from a list of data. The first row of the data is used as column names.
    The table is created using Snowpark, and any existing table with the same name is overwritten.

    Parameters:
    session: Snowpark session object.
    data: List of lists containing the data. The first list should contain column names.
    table_name: The name of the table to be created in Snowflake.
    """
    
    # Extract column names from the first row of data
    columns = data[0]
    data = data[1:]
    
    # Fix rows with missing columns by appending empty strings to match the length of the columns
    fixed_data = [row + [''] * (len(columns) - len(row)) for row in data]
    
    # Replace special characters and spaces in column names to make them compatible with Snowflake
    clean_columns = [col.replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_").replace("ç", "c").replace("ã", "a") for col in columns]
    
    # Create a Pandas DataFrame using the fixed data and cleaned column names
    df = pd.DataFrame(fixed_data, columns=clean_columns)
    
    # Convert the Pandas DataFrame to a Snowpark DataFrame
    snowpark_df = session.create_dataframe(df)
    
    # Insert the data into Snowflake and overwrite any existing table with the same name
    snowpark_df.write.mode("overwrite").save_as_table(table_name)

    # Print a message indicating the table was created and show the first few rows of the table
    print('Table created:')
    df_table = session.table(table_name)
    df_table.show()

In [ ]:
create_table(session, csc_data, table_name='teste_csc')

In [39]:
create_table(session, tlog_data, table_name='teste_tlog')

Table created:
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Sprint"  |"ID"   |"CTI"  |"Classificacao"  |"Task"                                            |"Descricao"  |"Entrega"                       |"Produto"  |"Responsavel"                   |"Marcador"  |"Funcao"               |"Bitrix?"  |"Status"   |"Inicio"    |"Conclusao"  |"Tempo_Estimado_horas"  |"Tempo_Efetivo_horas"  |"Tempo_Efetivo_Repasse_horas"  |"Id_task_atrasada"  |"Previsto_no_Planejamento?"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
create_table(session, pmo_data, table_name='teste_pmo')

Table created:
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Sprint"  |"ID"    |"CTI"        |"Classificacao"  |"Task"              |"Descricao"  |"Entrega"            |"Produto"  |"Responsavel"               |"Marcador"  |"Funcao"               |"Bitrix?"  |"Status"   |"Inicio"    |"Conclusao"  |"Tempo_Estimado_horas"  |"Tempo_Efetivo_horas"  |"Tempo_Efetivo_Repasse_horas"  |"Id_task_atrasada"  |"Previsto_no_Planejamento?"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
create_table(session, engagement_survey_data, table_name='teste_survey')

Table created:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Carimbo_de_data/hora"  |"Endereco_de_e_mail"            |"Sprint_de_referência_devops"  |"Em_qual_squad_você_está_atuando?"  |"Me_senti_engajado_para_realizar_minhas_ativida...  |"Sinto_que_me_desenvolvi_profissionalmente_ao_l...  |"Sinto_que_conclui_as_atividades_da_sprint_com_...  |"Sinto_que_conclui_as_atividades_da_sprint_no_p...  |"De_forma_geral,_me_sinto_bem_e_satisfeito_trab...  |"Deixe_seu_comentário_ou_sugestao_do_que_podemo...  |
-------------------------------------------------------------------------------------------

# Feedback

In [18]:
create_table_sql = """
CREATE OR REPLACE TABLE feedback (
    timestamp STRING,
    email STRING,
    sprint_ref STRING,
    squad STRING,
    engagement INT,
    development INT,
    quality INT,
    on_time INT,
    satisfaction INT,
    comments STRING
)
"""

session.sql(create_table_sql).collect()

---------------------------------------------------------------------------------------------------------------------------------------
|"TIMESTAMP"  |"EMAIL"  |"SPRINT_REF"  |"SQUAD"  |"ENGAGEMENT"  |"DEVELOPMENT"  |"QUALITY"  |"ON_TIME"  |"SATISFACTION"  |"COMMENTS"  |
---------------------------------------------------------------------------------------------------------------------------------------
|             |         |              |         |              |               |           |           |                |            |
---------------------------------------------------------------------------------------------------------------------------------------



In [19]:
for row in sheet_data[1:]:
    session.sql(f"""
    INSERT INTO feedback (timestamp, email, sprint_ref, squad, engagement, development, quality, on_time, satisfaction, comments)
    VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', {row[4]}, {row[5]}, {row[6]}, {row[7]}, {row[8]}, '{row[9]}')
    """).collect()

In [20]:
# Inserir os dados
for row in sheet_data[1:]:
    values = "', '".join(row)  # Preparar os valores para inserção
    session.sql(f"""
    INSERT INTO feedback ({', '.join(column_names_sql)})
    VALUES ('{values}')
    """).collect()

NameError: name 'column_names_sql' is not defined

In [ ]:
customer_wrt = df_table.write.mode("overwrite").save_as_table("SANDBOX.DEV_IVON_GARCIA.feedback")

In [ ]:
column_names = sheet_data[0]
column_names

In [ ]:
column_names_sql = [name.replace(' ', '_').replace('(', '').replace(')', '').replace('\n', '') for name in column_names]


In [ ]:
column_names_sql

In [ ]:
# from snowflake import snowpark
# from config import *
# SNOWFLAKE_CONFIG = {
#         "account": account_id,
#         "user": username,
#         "password": password,
#         "role": role,
#         "warehouse": warehouse,
#         "database": database,
#         "schema": schema}
# connection = snowpark.Session(SNOWFLAKE_CONFIG)
# # write_pandas(conn=connection, df=sheet_df, table_name='teste_kpi')

In [ ]:
pandas_df

In [ ]:
snowpark_df = session.write_pandas(conn=session, df=pandas_df, table_name="asas", auto_create_table=True, table_type="temp")

In [ ]:
snowpark_df.sort('"city"').to_pandas()

In [ ]:
def write_dataframe(session, df, table_name):
        success, nchunks, nrows, _ = write_pandas(session, df, table_name)
        if success:
            print(f"Successfully inserted {nrows} rows into {table_name}")
        else:
            print("Failed to insert data")

# Escrever dados no Snowflake
if sheet_data is not None:
    # Certifique-se de que o DataFrame tem as colunas corretas e tipos de dados
    sheet_data.columns = sheet_data.iloc[0]
    # sheet_data = sheet_data.drop(0)
    
    write_dataframe(session, sheet_data, "YOUR_TABLE_NAME")
    session.close()
else:
    print("No data to write to Snowflake")